#  Submit Molecule on Slab Geometry Optimization

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.code import Code
from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.base import Int, Float, Str, Bool
from aiida.work.run import submit
from aiida.orm.data.structure import StructureData

import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from IPython.display import display, clear_output, HTML
import nglview

from apps.surfaces.structure_browser import StructureBrowser
from apps.surfaces.slab.slabwork import SlabGeoOptWorkChain

In [ ]:
def on_struct_change(c):
    refresh_structure_view()
    on_dft_toggle(v={'new': ''})
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()

clear_output()
display(ipw.VBox([struct_browser, viewer]))

In [ ]:
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    node = struct_browser.results.value
    if not node:
        return
    atoms = node.get_ase()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()

    spin_ups = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("1")])
    spin_downs = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("2")])

    viewer.add_representation('ball+stick', selection="@"+spin_ups, color='red', aspectRatio=3.0, opacity=0.4)
    viewer.add_representation('ball+stick', selection="@"+spin_downs, color='green', aspectRatio=3.0, opacity=0.4)

In [ ]:
def on_dft_toggle(v):
    try:
        atoms = struct_browser.results.value.get_ase()

        first_slab_atom, last_slab_atom = np.argwhere(atoms.numbers == 79)[0, 0], len(atoms)

        slab = atoms.get_atomic_numbers()[first_slab_atom:last_slab_atom]
        slab_h = np.count_nonzero(slab==1)
        slab_three_rows = slab_h*3

        first_fixed_atom = len(atoms)-slab_three_rows
        last_fixed_atom = len(atoms)
    
        dft_type = toggle_dft.value

        if dft_type in ['Mixed DFTB', 'Mixed DFT']:
            text_fixed_atoms.value = '%d..%d'%(first_slab_atom+1, last_slab_atom)
        if dft_type in ['Full DFT']:
            text_fixed_atoms.value = '%d..%d'%(first_fixed_atom, last_fixed_atom)
    except:
        pass
    
    with dft_opt:
        style = {'description_width': '120px'}
        layout = {'width': '70%'}
        clear_output()
        if v['new'] in ['Mixed DFT', 'Full DFT']:
            display(slider_cutoff)


In [ ]:
# query AiiDA database for Computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'cp2k'})

all_computers = OrderedDict()
all_computers['Please select a computer'] = False
for match in qb.all():
    label = match[0]
    code = match[1]
    all_computers[label] = code

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer", options=all_computers,
                            style=style, layout=layout)

text_fixed_atoms = ipw.Text(placeholder='1..12 25 110',
                            description='Fixed Atoms',
                            disabled=True,
                            style=style, layout=layout)

slider_precision = ipw.BoundedFloatText(description='MAX_FORCE:', value=1e-3, min=1e-4, max=1e-3, step=1e-4,
                                  style=style, layout=layout)
toggle_dft = ipw.ToggleButtons(options=['Mixed DFTB', 'Mixed DFT', 'Full DFT'],
                               description='Calculation Type',
                               tooltip='Active: DFT, Inactive: DFTB',
                               style=style, layout=layout)

slider_cutoff = ipw.IntSlider(description='MGRID_CUTOFF:',
                              value=600, step=100,
                              min=100, max=1200,
                              style=style, layout=layout)

toggle_vdw = ipw.ToggleButton(value=False,
                              description='Dispersion Corrections',
                              tooltip='VDW_POTENTIAL',
                              style=style, layout=layout)

toggle_center = ipw.ToggleButton(value=False,
                              description='Center Coordinates (only FULL DFT)',
                              tooltip='Center Coordinates',
                              style=style, layout=layout)

toggle_dft.observe(on_dft_toggle, 'value')
dft_opt = ipw.Output()

display(ipw.VBox([drop_computer, toggle_dft, text_fixed_atoms, slider_precision, toggle_vdw, toggle_center, dft_opt]))

In [ ]:
def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not drop_computer.value:
            print("Please select a computer.")
            return

        cp2k_code = drop_computer.value
        struct = struct_browser.results.value
        max_force = Float(slider_precision.value)
        calc_type = Str(toggle_dft.value)
        mgrid_cutoff = Int(slider_cutoff.value)
        vdw_switch = Bool(toggle_vdw.value)
        fixed_atoms = Str(text_fixed_atoms.value)
        center_switch = Bool(toogle_center.value)
        outputs = submit(SlabGeoOptWorkChain,
                         cp2k_code=cp2k_code,
                         structure=struct,
                         max_force=max_force,
                         calc_type=calc_type,
                         mgrid_cutoff=mgrid_cutoff,
                         vdw_switch=vdw_switch,
                         fixed_atoms=fixed_atoms,
                         center_switch=center_switch)
        print(outputs)



btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)

In [ ]:
def on_debug():
    slabwork = SlabGeoOptWorkChain()

    cp2k_code = drop_computer.value
    struct = struct_browser.results.value
    max_force = Float(slider_precision.value)
    calc_type = Str(toggle_dft.value)
    mgrid_cutoff = Int(slider_cutoff.value)
    vdw_switch = Bool(toggle_vdw.value)
    fixed_atoms = Str(text_fixed_atoms.value)

    atoms = struct.get_ase()  # slow
    cell_abc = "%f  %f  %f" % (atoms.cell[0, 0], atoms.cell[1, 1], atoms.cell[2, 2])
    first_slab_atom = np.argwhere(atoms.numbers == 79)[0, 0] + 1
    last_slab_atom = len(atoms)
    
    machine_cores = cp2k_code.get_remote_computer().get_default_mpiprocs_per_machine()
    machines_cores = (1. + first_slab_atom/30.)*machine_cores


        
    # inp = slabwork.build_calc_inputs(struct, cp2k_code, max_force, dftb_switch, mgrid_cutoff, vdw_switch)

    inp = slabwork.get_cp2k_input(cell_abc=cell_abc,
                                   first_slab_atom=first_slab_atom,
                                   last_slab_atom=last_slab_atom,
                                   max_force=max_force,
                                   calc_type=calc_type,
                                   mgrid_cutoff=mgrid_cutoff,
                                   vdw_switch=vdw_switch,
                                   machine_cores = machines_cores,
                                   fixed_atoms = fixed_atoms)

    import pprint

    pprint.pprint(inp)

    with open('new_inp'+'_mixedDFTB'+'.inp', 'w') as f:
        f.write(pprint.pformat(inp))

    from IPython.display import FileLink
    from datetime import datetime
    FileLink('new_inp'+'_mixedDFTB'+'.inp')
    
#print on_debug()